In [9]:
!pip install nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.8 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [19]:
pip install beautifulsoup4



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import re
from gensim.parsing.preprocessing import STOPWORDS, strip_tags,strip_numeric, strip_punctuation, strip_multiple_whitespaces,remove_stopwords, strip_short, stem_text
import pickle
from transformers import BertTokenizer
import nltk


In [22]:
import numpy as np

In [38]:
import re
import string
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shashinimashi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
from nltk.corpus import stopwords

In [15]:
# Load the CSV file
file_path = '/Users/shashinimashi/Desktop/Semester 3/Thesis/Analysis/Research_OAI-1/transcripts.csv'  
df = pd.read_csv(file_path)

In [16]:
df.head()

,ResponseID,Topic,Transcription
0,510,Supporting Responsible Ai Discussion Paper,The Australian Federal Police Afp Welcomes The...
1,507,Safe And Responsible Ai In Australia’,The Disr Have Called For Public Submissions To...
2,506,Safe And Responsible Ai In Australia\r\nDiscus...,The Office Of The Australian Information Commi...
3,505,Canvas Approach To Ai,Canvas Mission Is To Empower The World To Desi...
4,504,Safe And Responsible Ai In\r\nAustralia,The Law Council Welcomes The Opportunity To Pr...


In [18]:
df.columns

Index(['ResponseID', 'Topic', 'Transcription'], dtype='object')

In [21]:
df.value_counts

<bound method DataFrame.value_counts of      ResponseID                                              Topic  \
0           510         Supporting Responsible Ai Discussion Paper   
1           507              Safe And Responsible Ai In Australia’   
2           506  Safe And Responsible Ai In Australia\r\nDiscus...   
3           505                              Canvas Approach To Ai   
4           504            Safe And Responsible Ai In\r\nAustralia   
..          ...                                                ...   
411           7  Epistemic Virtues Of Harnessing Rigorous\r\nMa...   
412           6  Ai Explainability Framework For Environmental ...   
413           4  Submission To Responsible Ai In Australia – Ga...   
414           2  Ben Blackburn Racing Submission\r\nOn The Aust...   
415           1                             Make A General Comment   

                                         Transcription  
0    The Australian Federal Police Afp Welcomes The...  
1    

In [51]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shashinimashi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [53]:
import re
import string
from transformers import BertTokenizer
from bs4 import BeautifulSoup
import nltk
import pandas as pd  

# Download NLTK data for sentence tokenization (ensure 'punkt' is available)
nltk.download('punkt')

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def clean_and_tokenize(df, column_name='Transcription'):
    
    # Ensure the column exists in the DataFrame
    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the DataFrame.")
        return df
    
    def clean_text(text):
        if not text or pd.isna(text):  # Check for empty or NaN values
            return []  # Return an empty list if the text is invalid

        # Lowercase the text
        text = text.lower()
        
        # Remove URLs
        text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
        
        # Remove HTML tags
        text = BeautifulSoup(text, "html.parser").get_text()
        
        # Remove punctuation
        text = text.translate(str.maketrans("", "", string.punctuation))
        
        # Remove special characters and numbers
        text = re.sub(r"[^a-zA-Z\s]", "", text)
        
        # Tokenize the text using BERT tokenizer (subword-level)
        tokens = tokenizer.tokenize(text)
        
        # Rejoin tokens into a single string
        cleaned_text = " ".join(tokens)
        
        # Alternative sentence tokenization using re.split()
        sentences = re.split(r'(?<=[.!?])\s+', cleaned_text)
        
        return sentences

    # Debug: Print first few rows before cleaning
    print(f"First few rows of the column '{column_name}':")
    print(df[column_name].head())
    
    # Apply the clean_text function to each entry in the column
    df['Responses'] = df[column_name].apply(clean_text)
    
    # Debug: Check if the Responses column has been added
    if 'Responses' in df.columns:
        print(f"'Responses' column has been created successfully.")
    else:
        print(f"Error: 'Responses' column not created.")
    
    # Print the DataFrame with Transcriptions and Responses
    print(df[['Transcription', 'Responses']].head())
    
    return df

# Example usage:
df_cleaned = clean_and_tokenize(df)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shashinimashi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


First few rows of the column 'Transcription':
0    The Australian Federal Police Afp Welcomes The...
1    The Disr Have Called For Public Submissions To...
2    The Office Of The Australian Information Commi...
3    Canvas Mission Is To Empower The World To Desi...
4    The Law Council Welcomes The Opportunity To Pr...
Name: Transcription, dtype: object
'Responses' column has been created successfully.
                                       Transcription  \
0  The Australian Federal Police Afp Welcomes The...   
1  The Disr Have Called For Public Submissions To...   
2  The Office Of The Australian Information Commi...   
3  Canvas Mission Is To Empower The World To Desi...   
4  The Law Council Welcomes The Opportunity To Pr...   

                                           Responses  
0  [the australian federal police af ##p welcome ...  
1  [the di ##sr have called for public submission...  
2  [the office of the australian information comm...  
3  [canvas mission is to em ##power t

In [54]:
df.head()

,ResponseID,Topic,Transcription,Responses
0,510,Supporting Responsible Ai Discussion Paper,The Australian Federal Police Afp Welcomes The...,[the australian federal police af ##p welcome ...
1,507,Safe And Responsible Ai In Australia’,The Disr Have Called For Public Submissions To...,[the di ##sr have called for public submission...
2,506,Safe And Responsible Ai In Australia\r\nDiscus...,The Office Of The Australian Information Commi...,[the office of the australian information comm...
3,505,Canvas Approach To Ai,Canvas Mission Is To Empower The World To Desi...,[canvas mission is to em ##power the world to ...
4,504,Safe And Responsible Ai In\r\nAustralia,The Law Council Welcomes The Opportunity To Pr...,[the law council welcome ##s the opportunity t...
